In [ ]:
import numpy as np
import scipy
from scipy.linalg import sqrtm
import matplotlib.pyplot as plt
from AnsatzLacs import *
from CCAUtil import *

In [ ]:
# implementation of the slack variable method
# quantum circuit settings
# these values will be used as global variables later in different functions
A_size = 1 # size of the A subsystem input state (2 means 2 qubit)
B_size = 1 # size of the B subsystem input state (2 means 2 qubit)
sys_size = A_size + B_size
ref_size = 0 
num_coefs = 16
full_size = 4 ** sys_size
problem_seed = 3

In [ ]:
def partial_transpose(rho):
    """
    Returns the partial transpose of rho (bipartite) with respect to the second subsystem.

    param rho: square density matrix describing bipartite state
    return: square desity matrix describing partially transposed bipartite state
    """
    dim1, dim2 = rho.shape
    dimA = int(np.sqrt(dim1))
    
    # Split rows
    split1 = np.split(rho,(np.arange(dimA-1)+1)*dimA, axis=0)
    
    # Split columns
    split2 = []
    for sec in split1:
        split2.extend(np.split(sec,(np.arange(dimA-1)+1)*dimA, axis=1))
    
    # Transpose subsystem B
    split2_trans = []
    for sec in split2:
        split2_trans.append(sec.T)
    
    # Concatinate back together
    rho_trans = []
    for i in range(dimA):
        for k in range(dimA):
            row = []
            for j in range(dimA):
                row.extend(split2_trans[i*dimA+j][k])
            rho_trans.append(row)
    
    return(rho_trans)

def negativity(rho):
    """
    Returns the entanglement negativity of rho.

    param rho: square density matrix describing bipartite state
    return: floating point number between 0 and .5
    """
    # Perform the partial transpose of rho
    rho_trans = partial_transpose(rho)
    
    # Sum the negative eigenvalues of the partial transpose of rho
    eigenvals, eigenvecs = np.linalg.eig(rho_trans)
    eigenvals[eigenvals>0] = 0
    sum_neg_eigenvals = np.abs(np.sum(eigenvals))
    return 2 * sum_neg_eigenvals + 1

In [ ]:
np.random.seed(problem_seed)
rho_ansatz = General(sys_size, ref_size, 2)
rho_prob = QCBM(sys_size, ref_size, 2)

rho_param = np.random.normal(size=rho_ansatz.get_parameter_num()) % (2 * np.pi)
rho_prob_param = np.random.normal(size=rho_prob.get_parameter_num()) % (2 * np.pi)

rho_ansatz.set_params(rho_param)
rho_prob.set_params(rho_prob_param)

rho = CCA(rho_prob, rho_ansatz)
expected = negativity(rho.get_dm(None))
print(f"Exact value: {expected}")

In [ ]:
def int_to_pauli_string(x):
    return ("" if x < 4 else int_to_pauli_string(x//4)) + 'IXYZ'[x % 4]

x = list(range(4**sys_size))
pauli_strings = []
for i in range(4**sys_size):
    string = int_to_pauli_string(x[i])
    string = (sys_size-len(string))*'I' + string
    pauli_strings.append(string)
    
expectation_unitary_dict = {pauli_string: get_expectation_unitary(pauli_string) for pauli_string in pauli_strings}
swap_test_unitary = get_swap_unitary(sys_size)

In [ ]:
def get_exp(pauli_string, ansatz, qcbm_shots, shots):
    return get_exp_helper(expectation_unitary_dict, pauli_string, ansatz, qcbm_shots, shots)

In [ ]:
# customized SPSA optimizer
def get_gradient(params, fun):
    delta = 1 - 2 * np.random.binomial(1, 0.5, size=len(params))
    plus = fun(params + perturbation * delta)
    minus = fun(params - perturbation * delta)
    grad = (plus - minus) / (2 * perturbation * delta)

    norm = np.linalg.norm(grad)
    if norm > 1:
        grad = grad/norm
    
    return grad

def gradient_descent(fun, x0, jac, maxiter, callback = None):
    x = x0
    
    for i in range(maxiter):
        iteration = i + 1
        
        x -= lr * jac(x, fun)
        
        if callback != None:
            callback(x, iteration)
    return x

def adam(fun, x0, jac, maxiter, beta1 = .9, beta2 = .99, callback = None):
    x = x0
    m = np.zeros(len(x))
    v = np.zeros(len(x))
    
    for i in range(maxiter):
        derivative = jac(x0, fun)
        
        m = beta1 * m + (1 - beta1) * derivative
        v = beta2 * v + (1 - beta2) * derivative * derivative
        lr_eff = lr * np.sqrt(1 - beta2**(i+1)) / (1 - beta1**(i+1))
        x -= lr_eff * m / np.sqrt(v)
    
        if callback != None:
            callback(x, i+1)
    return x

# Dual Optimization

In [ ]:
N = 2**sys_size
def get_params_dual(params):
    """
    [α, β, λ, μ, σ, τ]
    """
    alpha = np.array(params[:full_size])
    beta = np.array(params[full_size:2 * full_size])
    lmbda = (params[2 * full_size]) ** 2
    mu = (params[2 * full_size + 1]) ** 2
    current_index = 2 * full_size + 2
    param_num = tau.get_parameter_num()
    tau_params = params[current_index:current_index + param_num]
    sigma_params = params[current_index + param_num:current_index + 2 * param_num]
    return alpha, beta, lmbda, mu, tau_params, sigma_params

def dual_cost_function_full(params):
    alpha, beta, lmbda, mu, tau_params, sigma_params = get_params_dual(params)
    tau.set_params(tau_params)
    sigma.set_params(sigma_params)
    
    n = sys_size
    
    # Objective: Tr[K_AB + L_AB]
    obj = N * (alpha[0] + beta[0])
    
    # Penalty 1: ||T_B(K_AB - L_AB) - rho_AB||^2
    p1a = N * (np.sum(alpha ** 2) + np.sum(beta ** 2))
    p1b = get_purity(rho, qcbm_shots)
    
    tr_vec = np.zeros(full_size)
    for i in range(full_size):
        tr = get_exp(pauli_strings[i],rho, qcbm_shots, num_shots)
        if pauli_strings[i][1] == 'Y':
            tr_vec[i] = -tr
        else:
            tr_vec[i] = tr
        
    p1c = -2 * np.dot(alpha-beta,tr_vec)
    p1d = -2 * N * np.dot(alpha,beta)
    p1 = p1a + p1b + p1c + p1d
    
    # Penalty 2: ||K_AB - lmbda*sigma_AB||^2
    p2a = N * np.sum(alpha ** 2)

    tr_vec = np.zeros(full_size)
    for i in range(full_size):
        tr = get_exp(pauli_strings[i],sigma, qcbm_shots, num_shots)
        tr_vec[i] = tr
        
    p2b = -2 * lmbda * np.dot(alpha,tr_vec)
    p2c = (lmbda ** 2) * get_purity(sigma, qcbm_shots)
    p2 = p2a + p2b + p2c
    
    
    # Penalty 3: Tr[||L_AB - mu*tau_AB||^2]
    
    p3a = N * np.sum(beta ** 2)
    
    tr_vec = np.zeros(full_size)
    for i in range(full_size):
        tr = get_exp(pauli_strings[i],tau, qcbm_shots, num_shots)
        tr_vec[i] = tr
        
    p3b = -2 * mu * np.dot(beta,tr_vec)
    p3c = (mu ** 2) * get_purity(tau, qcbm_shots)
    p3 = p3a + p3b + p3c
    
    penalty = p1 + p2 + p3
    
    # Put all of it together
    cost = obj + c * penalty
   
    return cost, obj, penalty

def dual_cost_function(params):
    cost, obj, penalty = dual_cost_function_full(params)
    return cost

In [ ]:
print_every = 100
def dual_callback(x, iteration):
    global c
    global lr
    global lr_wait
    
    fx, obj, penal = dual_cost_function_full(x)

    dual_iterations.append(iteration)
    dual_loss.append(fx)
    dual_penals.append(penal)
    dual_objectives.append(obj)

    
    if (iteration%print_every==0):
        lr_wait += 1
        if lr_wait >= 5:
            m,_ = np.polyfit(dual_iterations[-500:], dual_loss[-500:], 1)
            if m > 0:
                lr = max(lr/2,lr_min)
                lr_wait = 0
        
        print(f"Iteration: {iteration}", "Loss: {:.4g}".format(fx), "Objective: {:.3g}".format(obj),
          "Penalty: {:.3g}".format(penal), "C: {:.2g}".format(c), "LR: {:.2g}".format(lr), sep = '\t')

In [ ]:
num_shots = 1e12
qcbm_shots = 1e12

max_iter = 100 * 1000
lr_max = 1e-1
lr_min = 1e-2
perturbation = 1e-4

dual_num_layers = 3
dual_qcbm_layers = 2

In [ ]:
num_train = 10

iterations_dual_outer = []
costs_dual_outer = []
objs_dual_outer = []
penalties_dual_outer = []

record = 0
for i in range(num_train):
    lr = lr_max
    c = 100
    lr_wait = 0
    
    alpha0 = np.zeros(4**sys_size)
    beta0 = np.zeros(4**sys_size)
    lmbda0 = 1.
    mu0 = 1.
    
    np.random.seed(i)
    tau = create_cca("general", sys_size, ref_size, dual_num_layers, dual_qcbm_layers)
    sigma = create_cca("general", sys_size, ref_size, dual_num_layers, dual_qcbm_layers)

    print("Expected value:", expected)
    init_params = [*alpha0, *beta0, np.sqrt(lmbda0), np.sqrt(mu0), *tau.get_params(), *sigma.get_params()]
    init_cost, init_obj, init_penal = dual_cost_function_full(init_params)
    
    dual_loss = [init_cost]
    dual_penals = [init_penal]
    dual_objectives = [init_obj]
    dual_iterations = [0]
    
    print(f"Iteration {i}, Initial cost : {init_cost}")
    result = gradient_descent(dual_cost_function, init_params, get_gradient, max_iter, callback=dual_callback)
    
    costs_dual_outer.append(dual_loss)
    objs_dual_outer.append(dual_objectives)
    penalties_dual_outer.append(dual_penals)
    iterations_dual_outer.append(dual_iterations)
    
    if abs(dual_loss[-1] - expected) <= 0.05:
        dir_name = f"Negativity_CCA_dual_data/seed{problem_seed}"
        problem_name = "Neg"
        filename = f"Init_{i}"
        save_data_to_file(dual_loss, dual_penals, dual_objectives, problem_name, dir_name, filename)
        record += 1
        
print("Number of successful trails: ", record)

# Primal Optimization

In [ ]:
def get_params_primal(params):
    """
    [α, λ, μ, τ, σ]
    """
    alpha = params[:full_size]
    lmbda = (params[full_size])**2
    mu = (params[full_size + 1])**2
    param_start = full_size + 2
    param_num = tau.get_parameter_num()
    tau_params = params[param_start:param_start + param_num]
    sigma_params = params[param_start + param_num:param_start + 2 * param_num]
    return alpha, lmbda, mu, tau_params, sigma_params

def primal_cost_function_full(params):
    alpha, lmbda, mu, tau_params, sigma_params = get_params_primal(params)
    
    tau.set_params(tau_params)
    sigma.set_params(sigma_params)
    
    n = sys_size
    
    # Term 1: Tr[T_B(H_AB) rho_AB]
    tr_vec = np.zeros(full_size)
    for i in range(full_size):
        tr = get_exp(pauli_strings[i],rho, qcbm_shots, num_shots)
        if pauli_strings[i][1] == 'Y':
            tr_vec[i] = -tr
        else:
            tr_vec[i] = tr
        
    t1 = np.dot(alpha,tr_vec)
    
    # Term 3: Tr[H_AB^2]
    t3 = N * np.sum(np.square(alpha))
    
    # Term 4: Tr[H_AB sigma_AB]
    tr_vec = np.zeros(full_size)
    for i in range(full_size):
        tr = get_exp(pauli_strings[i],sigma, qcbm_shots, num_shots)
        tr_vec[i] = tr
        
    t4 = np.dot(alpha,tr_vec)
    
    # Term 5: Tr[H_AB tau_AB]
    tr_vec = np.zeros(full_size)
    for i in range(full_size):
        tr = get_exp(pauli_strings[i],tau, qcbm_shots, num_shots)
        tr_vec[i] = tr
        
    t5 = np.dot(alpha,tr_vec)
    
    # Term 6: Tr[sigma_AB^2]
    t6 = get_purity(sigma, qcbm_shots)
    
    # Term 7: Tr[tau_AB^2]
    t7 = get_purity(tau, qcbm_shots)
    
    # Term 8: Tr[I_AB]
    t8 = N
    

    # Put all of it together
    obj = t1
    
    penalty = 2 * (t8 + t3 - lmbda - mu + lmbda * t4 - mu * t5) + (lmbda ** 2) * t6 + (mu ** 2) * t7
    cost = obj - c * penalty
   
    return cost, obj, penalty

def primal_cost_function(params):
    cost, obj, penalty = primal_cost_function_full(params)
    return -cost

In [ ]:
print_every = 100
def primal_callback(x, iteration):
    global c
    global lr
    global lr_wait
    
    fx, obj, penal = primal_cost_function_full(x)

    primal_iterations.append(iteration)
    primal_loss.append(fx)
    primal_penals.append(penal)
    primal_objectives.append(obj)

    
    if (iteration%print_every==0):
        lr_wait += 1
        if lr_wait >= 5:
            m,_ = np.polyfit(primal_iterations[-500:], primal_loss[-500:], 1)
            if m < 0:
                lr = max(lr/2,lr_min)
                lr_wait = 0
        
        print(f"Iteration: {iteration}", "Loss: {:.4g}".format(fx), "Objective: {:.3g}".format(obj),
          "Penalty: {:.3g}".format(penal), "C: {:.2g}".format(c), "LR: {:.2g}".format(lr), sep = '\t')

In [ ]:
num_shots = 1e12
qcbm_shots = 1e12

max_iter = 100 * 1000
c = 10
lr_max = 1e-1
lr_min = 1e-2
perturbation = 1e-4

primal_num_layers = 2
primal_qcbm_layers = 1

In [ ]:
num_train = 3

iterations_primal_outer = []
costs_primal_outer = []
objs_primal_outer = []
penalties_primal_outer = []

record = 0
for i in range(num_train):
    lr = lr_max
    lr_wait = 0
    
    alpha0 = np.zeros(4**sys_size)
    lmbda0 = 1.
    mu0 = 1.
    
    np.random.seed(i)
    tau = create_cca("general", sys_size, ref_size, primal_num_layers, primal_qcbm_layers)
    sigma = create_cca("general", sys_size, ref_size, primal_num_layers, primal_qcbm_layers)

    print("Expected value:", expected)
    init_params = [*alpha0, np.sqrt(lmbda0), np.sqrt(mu0), *tau.get_params(), *sigma.get_params()]
    init_cost, init_obj, init_penal = primal_cost_function_full(init_params)
    
    primal_loss = [init_cost]
    primal_penals = [init_penal]
    primal_objectives = [init_obj]
    primal_iterations = [0]
    
    print(f"Iteration {i}, Initial cost : {init_cost}")
    result = gradient_descent(primal_cost_function, init_params, get_gradient, max_iter, callback=primal_callback)
    
    costs_primal_outer.append(primal_loss)
    objs_primal_outer.append(primal_objectives)
    penalties_primal_outer.append(primal_penals)
    iterations_primal_outer.append(primal_iterations)
    
    if abs(primal_loss[-1] - expected) <= 0.05:
        dir_name = f"Negativity_CCA_primal_data/seed{problem_seed}"
        problem_name = "Neg"
        filename = f"Init_{i}"
        save_data_to_file(primal_loss, primal_penals, primal_objectives, problem_name, dir_name, filename)
        record += 1
        
print("Number of successful trails: ", record)

# Plots

In [ ]:
# plt.rc('font', family='Times New Roman')
plt.rc('font', family='Arial')
plt.rc('axes', titlesize=24)
plt.rc('axes', labelsize=18)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=14)
plt.rc('legend', framealpha = 1)
plt.rc('legend', shadow = True)

In [ ]:
# Size of the smoothing window
window_size_primal = 500
window_size_dual = 500

iterations_primal = np.arange(window_size_primal, len(iterations_primal_outer[0])+1)
iterations_dual = np.arange(window_size_dual, len(iterations_dual_outer[0])+1)

costs_primal_smooth = get_smooth(costs_primal_outer[:-1], window_size_primal)
costs_dual_smooth = get_smooth(costs_dual_outer[:-1], window_size_dual)

In [ ]:
# IQR plot
max_iter_num = max(len(iterations_primal_outer[0]), len(iterations_dual_outer[0]))

costs_primal_median = np.median(costs_primal_smooth, axis = 0)
costs_primal_max = np.percentile(costs_primal_smooth, 75, axis = 0)
costs_primal_min = np.percentile(costs_primal_smooth, 25, axis = 0)

costs_dual_median = np.median(costs_dual_smooth, axis = 0)
costs_dual_max = np.percentile(costs_dual_smooth, 75, axis = 0)
costs_dual_min = np.percentile(costs_dual_smooth, 25, axis = 0)

# Plotting the line graph
expected_line = np.zeros(max_iter_num) + expected
plt.plot(iterations_primal, costs_primal_median, color = 'darkblue', linewidth = 1.5)
plt.plot(iterations_dual, costs_dual_median, color = 'purple', linewidth = 1.5)
plt.plot(np.arange(max_iter_num), expected_line, linestyle='dashed', color = 'deeppink', markersize = 1, linewidth = 2)

plt.legend(['QSlack Primal','QSlack Dual', 'True Value'])

plt.fill_between(iterations_primal, costs_primal_max, costs_primal_min, color = 'darkblue', alpha = .2)
plt.fill_between(iterations_dual, costs_dual_max, costs_dual_min, color = 'plum', alpha = .2)

plt.xlabel('Iteration')
plt.ylabel('Objective Function Value')
plt.grid(True)
# plt.ylim(-8, 3)
plt.xlim(0, max_iter_num)

# Display the graph
plt.show()